In [ ]:
import os, json
from xml.dom import minidom
import pandas as pd

source_path = 'D:/FNS_files_quantity_workers/source_files/' # папка с распакованными файлами ФНС
results_path = 'D:/FNS_files_quantity_workers/result_files/'

files = os.listdir(source_path)

In [ ]:
def parse_line(line): # извлечение одной компании
    date = line.attributes['ДатаСост'].value
    inn = line.getElementsByTagName('СведНП')[0].attributes['ИННЮЛ'].value
    name = line.getElementsByTagName('СведНП')[0].attributes['НаимОрг'].value
    personel = line.getElementsByTagName('СведССЧР')[0].attributes['КолРаб'].value
    return {'date': date, 'inn': inn, 'name': name, 'personel': personel}

def get_inn(line): # извлечение ИНН
    return line.getElementsByTagName('СведНП')[0].attributes['ИННЮЛ'].value

def parse_file(file): # распасить весь файл, file - один файл из набора
    mydoc = minidom.parse(file)
    lines = mydoc.getElementsByTagName('Документ')
    total = []
    for line in lines:
        total.append(parse_line(line))
    return total

def save_files(file_list, result_file_name):
                #file_list - все файлы в наборе в виде списка, result_file_name - имя распарсенного файла
    empty = []
    for file in file_list:
        empty.extend(parse_file(source_path + file))
    with open('%s%s.json' % (results_path, result_file_name), 'w') as outfile:
        json.dump(empty, outfile)

def filter_file(file): # сохранить JSON с информацией, у скольких компаний из файла 0-5 работников из распарсенного JSON
    with open(results_path + file, encoding='utf-8') as data_file: 
        data = json.loads(data_file.read())
    df = pd.DataFrame(data)
    df.personel = df.personel.astype('int64')
    zeros = df[df.personel == 0]
    ones = df[df.personel == 1]
    twos = df[df.personel == 2]
    threes = df[df.personel == 3]
    fours =  df[df.personel == 4]
    fives =  df[df.personel == 5]
    return {'total' : len(df), 'zeros' : len(zeros),
                               'ones' : len(ones),
                               'twos' : len(twos),
                               'threes' : len(threes),
                               'fours' : len(fours),
                               'fives' : len(fives)}

In [ ]:
save_files(files, 'result.json')

# сколько компаний с 0-5 работников
frames = filter_file(results_path + 'result.json')
pd.DataFrame(frames).to_csv('D:/FNS_files_quantity_workers/сколько всего компаний по 0-5 работников.csv')

In [ ]:
frames = []
for file in files:
    frames.extend(parse_file(source_path + file))
df = pd.DataFrame(frames)
df.personel = df.personel.astype('int64')

df[df.personel == 0].to_csv('D:\FNS_files_quantity_workers\все компании с числом сотрудников 0.csv')
df[df.personel > 5].sort_values('personel', ascending = False)\
                   .head(200).to_csv('D:\FNS_files_quantity_workers\200 компаний с наибольшим числом сотрудников.csv')

In [ ]:
# поиск имени файла, в котором находится определенная компания 
for file in files:
    mydoc = minidom.parse(source_path + file)
    lines = mydoc.getElementsByTagName('Документ')
    for line in lines:
        inn = get_inn(line)
        if inn == '1609029939':
            print(file)